In [ ]:
# Jupyter Notebook Cell 1: Imports
import os
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets
from src.utils.visualization_util import (
    plot_equity_curve, plot_drawdown, plot_daily_returns, plot_monthly_returns,
    plot_histogram_returns, plot_rolling_sharpe, plot_signals_on_price, plot_overlay_strategies
)
from src.utils.backtest_util import construct_strategy_hyperparam_str

# Set sim_dir here (use latest)
sim_dir = sorted([os.path.join("data/simulation_results", d) for d in os.listdir("data/simulation_results") if d.startswith("sim_")], key=lambda x: int(x.split('_')[-1]))[-1]
trade_dir = os.path.join(sim_dir, "trade")
metrics_path = os.path.join(sim_dir, "metrics_summary.csv")
metrics_df = pd.read_csv(metrics_path)

In [ ]:
# Jupyter Notebook Cell 2: Interactive Dropdowns for run selection
tokens = sorted(metrics_df['token'].unique())
strategies = sorted(metrics_df['strategy'].unique())
intervals = sorted(metrics_df['interval'].unique())

def show_run_plots(token, interval, strategy):
    subset = metrics_df[(metrics_df['token']==token) & (metrics_df['interval']==interval) & (metrics_df['strategy']==strategy)]
    if subset.empty:
        print("No runs for this combination.")
        return
    for _, row in subset.iterrows():
        key = f"{row['token']}_{row['interval']}_{row['strategy']}_{construct_strategy_hyperparam_str(row)}"
        trade_file = os.path.join(trade_dir, key + ".csv")
        if not os.path.exists(trade_file):
            continue
        # For demo: simulate equity_curve from trades
        trades = pd.read_csv(trade_file, parse_dates=['entry_time', 'exit_time'])
        eq = row.get('initial_capital', 1000000)
        equity_curve = []
        for idx, t in trades.iterrows():
            eq += t['pnl']
            equity_curve.append({'date': t['exit_time'], 'equity': eq})

        print(f"Run: {key}")
        plot_equity_curve(equity_curve, show=True, title="Equity Curve")
        plot_drawdown(equity_curve, show=True, title="Drawdown")
        plot_daily_returns(equity_curve, show=True, title="Daily Returns")
        plot_monthly_returns(equity_curve, show=True, title="Monthly Returns")
        plot_histogram_returns(equity_curve, show=True, title="Histogram of Daily Returns")
        plot_rolling_sharpe(equity_curve, show=True, title="Rolling Sharpe Ratio")

interact(
    show_run_plots,
    token=widgets.Dropdown(options=tokens, description='Token:'),
    interval=widgets.Dropdown(options=intervals, description='Interval:'),
    strategy=widgets.Dropdown(options=strategies, description='Strategy:')
);

In [ ]:
# Jupyter Notebook Cell 3: Top-N Runs Overlay
def plot_top_n_overlay(n=3, metric='sharpe'):
    top_n = metrics_df.sort_values(by=metric, ascending=False).head(n)
    equity_curves_dict = {}
    keys = []
    for _, row in top_n.iterrows():
        key = f"{row['token']}_{row['interval']}_{row['strategy']}_{construct_strategy_hyperparam_str(row)}"
        trade_file = os.path.join(trade_dir, key + ".csv")
        if os.path.exists(trade_file):
            trades = pd.read_csv(trade_file, parse_dates=['entry_time', 'exit_time'])
            eq = row.get('initial_capital', 1000000)
            equity_curve = []
            for idx, t in trades.iterrows():
                eq += t['pnl']
                equity_curve.append({'date': t['exit_time'], 'equity': eq})
            equity_curves_dict[key] = equity_curve
            keys.append(key)
    plot_overlay_strategies(equity_curves_dict, keys, save_path=None, title=f"Top {n} Runs Overlay")
    plt.show()

interact(
    plot_top_n_overlay,
    n=widgets.IntSlider(min=1, max=10, step=1, value=3, description='Top N:'),
    metric=widgets.Dropdown(options=['sharpe','total_return','cagr','win_rate','profit_factor'], description='Metric:')
);